In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from tensorflow.contrib import rnn
import random 
import collections
import time

In [3]:
logs_path='C:\\Users\\naveen chauhan\\Desktop\\mldata\\mlp\\RNN tensoflow\\'
def read_data(file_name):
    lines=open(file_name,'r')
    content=lines.readlines()
    content=[x.strip() for x in content]
    content=[word for i in range(len(content)) for word in content[i].split()]
    content=np.array(content)
    return content

In [4]:
file_name="belling_the_cat.txt"
training_data=read_data(logs_path+file_name)

In [12]:
#now build the real dataset
def build_dataset(data):
    count=collections.Counter(data).most_common()
    dictionary=dict()
    for cnt,_ in count:
        dictionary[cnt]=len(dictionary)
    #reverse dictionary
    reverse_dictionary=dict(zip(dictionary.values(),dictionary.keys()))
    return dictionary,reverse_dictionary

In [13]:
dictionary,reverse_dictionary=build_dataset(training_data)

In [15]:
#now initialize all tensorflow variable 
n_input=3
n_hidden=512
vocab_size=len(dictionary)

x=tf.placeholder(tf.float32,[None,n_input,1])
y=tf.placeholder(tf.float32,[None,vocab_size])

training_iters=50000
display_step=1000
learning_rate=0.001

weights={
    'out': tf.Variable(tf.random_normal([n_hidden,vocab_size]))
}
biases={
    'out':tf.Variable(tf.random_normal([vocab_size]))
}

In [18]:
#now we have defined all variables ,hence we define rnn function to train on model
def RNN(x,weights,biases):
    x=tf.reshape(x,[-1,n_input])
    x=tf.split(x,n_input,1)
    
    #now we train
    rnn_cell=rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    
    outputs,states=rnn.static_rnn(rnn_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],weights['out'])+biases['out']

In [19]:
pred=RNN(x,weights,biases)

In [22]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer=tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [29]:
#now start the session and do all the stuff
init=tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    step=0
    off_set=random.randint(0,n_input+1)
    end_offset=n_input+1
    acc_total=0
    loss_total=0
    while step<training_iters:
        if off_set> (len(training_data)-end_offset):
            off_set=random.randint(0,n_input+1)
        symbols_in_keys=[dictionary[str(training_data[i])] for i in range(off_set,off_set+n_input)]
        symbols_in_keys=np.reshape(symbols_in_keys,[-1,n_input,1])
        symbols_out_onehot=np.zeros([vocab_size],dtype=float)
        symbols_out_onehot[dictionary[str(training_data[off_set+n_input])]]=1.0
        symbols_out_onehot=np.reshape(symbols_out_onehot,[1,-1])
       
        _,acc,loss,one_hot_pred=session.run([optimizer,accuracy,cost,pred],feed_dict={x:symbols_in_keys,y:symbols_out_onehot})
        loss_total+=loss
        acc_total+=acc
        
        if (step+1)%display_step==0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total=0
            loss_total=0
            symbols_in=[training_data[i] for i in range(off_set,off_set+n_input)]
            symbols_out=training_data[off_set+n_input]
            symbols_out_pred=reverse_dictionary[int(tf.argmax(one_hot_pred,1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step+=1
        off_set+=(n_input+1)

Iter= 1000, Average Loss= 4.335814, Average Accuracy= 6.90%
['bell', 'the', 'cat'] - [?] vs [.]
Iter= 2000, Average Loss= 2.950046, Average Accuracy= 19.00%
['said', 'that', 'is'] - [all] vs [by]
Iter= 3000, Average Loss= 2.443958, Average Accuracy= 33.60%
['proposal', 'met', 'with'] - [general] vs [,]
Iter= 4000, Average Loss= 2.171529, Average Accuracy= 45.50%
['and', 'could', 'easily'] - [retire] vs [had]
Iter= 5000, Average Loss= 2.013835, Average Accuracy= 49.40%
['.', 'by', 'this'] - [means] vs [said]
Iter= 6000, Average Loss= 1.681325, Average Accuracy= 55.90%
['venture', ',', 'therefore'] - [,] vs [we]
Iter= 7000, Average Loss= 1.406817, Average Accuracy= 63.70%
['her', 'approach', ','] - [we] vs [we]
Iter= 8000, Average Loss= 1.165792, Average Accuracy= 68.80%
['and', 'treacherous', 'manner'] - [in] vs [all]
Iter= 9000, Average Loss= 1.063496, Average Accuracy= 72.00%
['agree', ',', 'said'] - [he] vs [he]
Iter= 10000, Average Loss= 1.125618, Average Accuracy= 68.00%
['he', 'th